# 튜토리얼

In [2]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI

model = ChatOpenAI(api_key="sk-")
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "당신은 {ability} 에 능숙한 어시스턴트입니다. 20자 이내로 응답하세요",
        ),
        # 대화 기록을 변수로 사용, history 가 MessageHistory 의 key 가 됨
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}"),  # 사용자 입력을 변수로 사용
    ]
)
runnable = prompt | model  # 프롬프트와 모델을 연결하여 runnable 객체 생성

- 메시지 기록을 관리하는 것은 대화형 애플리케이션 또는 복잡한 데이터 처리 작업에서 매우 중요
- 메시지 기록을 효과적으로 관리하려면 다음 두 가지 주요 요소 필요

1. Runnable: 주로 Retriever, Chain 과 같이 BaseChatMessageHistory 와 상호작용하는 runnable 객체
2. BaseChatMessageHistory의 인스턴스를 반환하는 호출 가능한 객체(callable):
    - 메시지 기록을 관리하기 위한 객체
    - 이 객체는 메시지 기록을 저장, 검색, 업데이트하는 데 사용됩
    - 메시지 기록은 대화의 맥락을 유지하고, 사용자의 이전 입력에 기반한 응답을 생성하는 데 필요

- 메시지 기록을 구현하는 데에는 여러 방법이 있으며, memory integrations 페이지에는 다양한 저장소 옵션과 통합 방법이 소개되어 있음

<두 가지 주요 방법>

1. 인메모리 ChatMessageHistory 사용
    - 이 방법은 메모리 내에서 메시지 기록을 관리. 
    - 주로 개발 단계나 간단한 애플리케이션에서 사용
    - 인메모리 방식은 빠른 접근 속도를 제공
    - 단점: 애플리케이션을 재시작할 때 메시지 기록이 사라지는 단점

2. RedisChatMessageHistory를 사용하여 영구적인 저장소 활용
    - Redis를 사용하는 방법은 메시지 기록을 영구적으로 저장 가능
    - Redis는 높은 성능을 제공하는 오픈 소스 인메모리 데이터 구조 저장소
        - 분산 환경에서도 안정적으로 메시지 기록을 관리 가능
    - 이 방법은 복잡한 애플리케이션 또는 장기간 운영되는 서비스에 적합

- 메시지 기록을 관리하는 방법을 선택할 때는 애플리케이션의 요구사항, 예상되는 트래픽 양, 메시지 데이터의 중요성 및 보존 기간 등을 고려해야 함. 
- 인메모리 방식은 구현이 간단하고 빠르지만, 데이터의 영구성이 요구되는 경우 Redis와 같은 영구적인 저장소를 사용하는 것이 더 적합할 수 있음.

# 휘발성 대화기록: 인메모리(In-Memory)

<채팅 기록이 메모리에 저장되는 간단한 예시>

- RunnableWithMessageHistory 설정 매개변수
    - runnable
    - BaseChatMessageHistory 이거나 상속받은 객체. ex) ChatMessageHistory
    - input_messages_key: chain 을 invoke() 할때 사용자 쿼리 입력으로 지정하는 key
    - history_messages_key: 대화 기록으로 지정하는 key

In [4]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}  # 세션 기록을 저장할 딕셔너리


# 세션 ID를 기반으로 세션 기록을 가져오는 함수
def get_session_history(session_ids: str) -> BaseChatMessageHistory:
    print(session_ids)
    if session_ids not in store:  # 세션 ID가 store에 없는 경우
        # 새로운 ChatMessageHistory 객체를 생성하여 store에 저장
        store[session_ids] = ChatMessageHistory()
    return store[session_ids]  # 해당 세션 ID에 대한 세션 기록 반환


with_message_history = (
    RunnableWithMessageHistory(  # RunnableWithMessageHistory 객체 생성
        runnable,  # 실행할 Runnable 객체
        get_session_history,  # 세션 기록을 가져오는 함수
        input_messages_key="input",  # 입력 메시지의 키
        history_messages_key="history",  # 기록 메시지의 키
    )
)

- input_messages_key: 최신 입력 메시지로 처리될 키 지정
- history_messages_key: 이전 메시지를 추가할 키 지정

- 다음의 코드를 보면 RunnableWithMessageHistory 의 초기값에 session_id 키를 Default 로 삽입하는 것을 볼 수 있으며, </br>이 코드로 인하여 RunnableWithMessageHistory 는 대화 스레드 관리를 session_id 로 한다는 것을 간접적으로 파악할 수 있음.

- 즉, 대화 스레드별 관리는 session_id 별로 구현함을 알 수 있음.

In [5]:
with_message_history.invoke(
    # 수학 관련 질문 "코사인의 의미는 무엇인가요?"를 입력으로 전달합니다.
    {"ability": "math", "input": "What does cosine mean?"},
    # 설정 정보로 세션 ID "abc123"을 전달합니다.
    config={"configurable": {"session_id": "abc123"}},
)

abc123


AIMessage(content='Cosine is a trigonometric function that represents the ratio of the adjacent side to the hypotenuse in a right triangle.', response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 47, 'total_tokens': 73}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-b367fe41-0d0d-4dd5-8d56-abd57f6fbb0c-0', usage_metadata={'input_tokens': 47, 'output_tokens': 26, 'total_tokens': 73})

-> 같은 session_id 를 입력하면 이전 대화 스레드의 내용을 가져오기 때문에 이어서 대화 가능

In [6]:
# 메시지 기록을 포함하여 호출합니다.
with_message_history.invoke(
    # 능력과 입력을 설정합니다.
    {"ability": "math", "input": "이전의 내용을 한글로 답변해 주세요."},
    # 설정 옵션을 지정합니다.
    config={"configurable": {"session_id": "abc123"}},
)

abc123


AIMessage(content='코사인은 직각삼각형에서 인접 변의 길이를 빗변의 길이로 나타내는 삼각함수입니다.', response_metadata={'token_usage': {'completion_tokens': 49, 'prompt_tokens': 98, 'total_tokens': 147}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-8e9183b7-22e1-429a-a25c-fdd6b16cca4d-0', usage_metadata={'input_tokens': 98, 'output_tokens': 49, 'total_tokens': 147})

-> 다른 session_id 를 지정하면 대화기록이 없기 때문에 답변을 제대로 수행X

In [8]:
# 새로운 session_id로 인해 이전 대화 내용을 기억하지 않습니다.
with_message_history.invoke(
    # 수학 능력과 입력 메시지를 전달합니다.
    {"ability": "math", "input": "이전의 내용을 한글로 답변해 주세요"},
    # 새로운 session_id를 설정합니다.
    config={"configurable": {"session_id": "def234"}},
)

def234


AIMessage(content='수학에 능숙한 어시스턴트입니다.', response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 58, 'total_tokens': 77}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-49bfc8b7-8ab4-449c-bd05-dcf5cee32d1f-0', usage_metadata={'input_tokens': 58, 'output_tokens': 19, 'total_tokens': 77})

-> session_id: def234 는 존재하지 않기 때문에 엉뚱한 답변을 하는 것을 확인 가능

- 메시지 기록을 추적하는 데 사용되는 구성 매개변수는 ConfigurableFieldSpec 객체의 리스트를 history_factory_config 매개변수로 전달하여 사용자 정의할 수 있음.

- history_factory_config 를 새로 설정하게 되면 기존 session_id 설정을 덮어쓰게 됨.

<아래 예시> 
- user_id와 conversation_id라는 두 가지 매개변수 사용

In [10]:
from langchain_core.runnables import ConfigurableFieldSpec

store = {}  # 빈 딕셔너리를 초기화합니다.


def get_session_history(user_id: str, conversation_id: str) -> BaseChatMessageHistory:
    # 주어진 user_id와 conversation_id에 해당하는 세션 기록을 반환합니다.
    if (user_id, conversation_id) not in store:
        # 해당 키가 store에 없으면 새로운 ChatMessageHistory를 생성하여 저장합니다.
        store[(user_id, conversation_id)] = ChatMessageHistory()
    return store[(user_id, conversation_id)]


with_message_history = RunnableWithMessageHistory(
    runnable,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history",
    history_factory_config=[  # 기존의 "session_id" 설정을 대체하게 됩니다.
        ConfigurableFieldSpec(
            id="user_id",  # get_session_history 함수의 첫 번째 인자로 사용됩니다.
            annotation=str,
            name="User ID",
            description="사용자의 고유 식별자입니다.",
            default="",
            is_shared=True,
        ),
        ConfigurableFieldSpec(
            id="conversation_id",  # get_session_history 함수의 두 번째 인자로 사용됩니다.
            annotation=str,
            name="Conversation ID",
            description="대화의 고유 식별자입니다.",
            default="",
            is_shared=True,
        ),
    ],
)

In [11]:
with_message_history.invoke(
    # 능력(ability)과 입력(input)을 포함한 딕셔너리를 전달합니다.
    {"ability": "math", "input": "what is cosine?"},
    # 설정(config) 딕셔너리를 전달합니다.
    config={"configurable": {"user_id": "123", "conversation_id": "abc"}},
)

AIMessage(content='Trigonometric function relates to the adjacent side over the hypotenuse in a right triangle.', response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 46, 'total_tokens': 65}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-7182da0c-11d9-4af3-94f0-c9e7b80f7cb5-0', usage_metadata={'input_tokens': 46, 'output_tokens': 19, 'total_tokens': 65})

In [12]:
answer = with_message_history.invoke(
    # 능력(ability)과 입력(input)을 포함한 딕셔너리를 전달합니다.
    {"ability": "math", "input": "이전의 답변을 한글로 작성해 주세요"},
    # 설정(config) 딕셔너리를 전달합니다.
    config={"configurable": {"user_id": "123", "conversation_id": "abc"}},
)

# 다양한 Key를 사용한 Runnable 을 사용한 예시

## 1. Messages 객체를 입력, dict 형태의 출력

메시지를 입력으로 받고 딕셔너리를 출력으로 반환하는 경우

- [중요]: input_messages_key="input" 을 생략 -> 입력으로 Message 객체를 넣도록 설정

In [18]:
from langchain_core.messages import HumanMessage
from langchain_core.runnables import RunnableParallel

# chain 생성
chain = RunnableParallel({"output_message": ChatOpenAI(api_key="sk-")})


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    # 세션 ID에 해당하는 대화 기록이 저장소에 없으면 새로운 ChatMessageHistory를 생성합니다.
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    # 세션 ID에 해당하는 대화 기록을 반환합니다.
    return store[session_id]


# 체인에 대화 기록 기능을 추가한 RunnableWithMessageHistory 객체를 생성합니다.
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    # 입력 메시지의 키를 "input"으로 설정합니다.(생략시 Message 객체로 입력)
    # input_messages_key="input",
    # 출력 메시지의 키를 "output_message"로 설정합니다. (생략시 Message 객체로 출력)
    output_messages_key="output_message",
)

# 주어진 메시지와 설정으로 체인을 실행합니다.
with_message_history.invoke(
    # 혹은 "what is the definition of cosine?" 도 가능
    [HumanMessage(content="what is the definition of cosine?")],
    config={"configurable": {"session_id": "abc123"}},
)

{'output_message': AIMessage(content='In mathematics, the cosine of an angle in a right-angled triangle is defined as the ratio of the length of the side adjacent to the angle to the length of the hypotenuse. It can also be defined as the x-coordinate of a point on the unit circle that is formed by the angle. The cosine function is denoted as cos and is commonly used in trigonometry to calculate angles and distances in geometric problems.', response_metadata={'token_usage': {'completion_tokens': 86, 'prompt_tokens': 14, 'total_tokens': 100}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-86429a88-9928-4dfa-af68-9f916b6e0c89-0', usage_metadata={'input_tokens': 14, 'output_tokens': 86, 'total_tokens': 100})}

In [19]:
with_message_history.invoke(
    # 이전의 답변에 대하여 한글로 답변을 재요청합니다.
    [HumanMessage(content="이전의 내용을 한글로 답변해 주세요!")],
    # 설정 옵션을 딕셔너리 형태로 전달합니다.
    config={"configurable": {"session_id": "abc123"}},
)

{'output_message': AIMessage(content='수학에서 직각삼각형의 각의 코사인은 해당 각에 인접한 변의 길이와 빗변의 길이의 비로 정의됩니다. 또한 각에 해당하는 단위 원상의 점의 x-좌표로 정의될 수도 있습니다. 코사인 함수는 cos로 표시되며, 기하학적 문제에서 각과 거리를 계산하는 데 트리곤메트리에 널리 사용됩니다.', response_metadata={'token_usage': {'completion_tokens': 142, 'prompt_tokens': 125, 'total_tokens': 267}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-45c9d4c7-01b5-4dd6-b831-8a14cb983a65-0', usage_metadata={'input_tokens': 125, 'output_tokens': 142, 'total_tokens': 267})}

## 2. Messages 객체를 입력, Messages 객체의 출력

- [중요]: output_messages_key="output_message" 생략 -> 출력으로 Message 객체 반환

In [21]:
with_message_history = RunnableWithMessageHistory(
    ChatOpenAI(api_key="sk-"),  # ChatOpenAI 언어 모델을 사용합니다.
    get_session_history,  # 대화 세션 기록을 가져오는 함수를 지정합니다.
    # 입력 메시지의 키를 "input"으로 설정합니다.(생략시 Message 객체로 입력)
    # input_messages_key="input",
    # 출력 메시지의 키를 "output_message"로 설정합니다. (생략시 Message 객체로 출력)
    # output_messages_key="output_message",
)

In [22]:
with_message_history.invoke(
    # 이전의 답변에 대하여 한글로 답변을 재요청합니다.
    [HumanMessage(content="코사인의 의미는 무엇인가요?")],
    # 설정 옵션을 딕셔너리 형태로 전달합니다.
    config={"configurable": {"session_id": "def123"}},
)

AIMessage(content='코사인은 삼각함수 중 하나로, 직각삼각형에서의 각도에 대한 변화를 나타내는 함수입니다. 코사인은 직각삼각형에서 빗변과 인접변의 길이의 비율을 나타내며, 주어진 각도에 대한 코사인 값은 삼각형에서의 해당 변들의 길이에 의해 결정됩니다. 코사인 함수는 주기적이며, -1과 1 사이의 값을 가집니다.', response_metadata={'token_usage': {'completion_tokens': 156, 'prompt_tokens': 24, 'total_tokens': 180}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-0358a6df-6bcb-4adc-a660-f59d78b1aeb0-0', usage_metadata={'input_tokens': 24, 'output_tokens': 156, 'total_tokens': 180})

## 3. 모든 메시지 입력과 출력을 위한 단일 키를 가진 Dict

모든 입력 메시지와 출력 메시지에 대해 단일 키를 사용하는 방식

- itemgetter("input_messages")를 사용하여 입력 메시지 추출

In [23]:

from operator import itemgetter

with_message_history = RunnableWithMessageHistory(
    # "input_messages" 키를 사용하여 입력 메시지를 가져와 ChatOpenAI()에 전달합니다.
    itemgetter("input_messages") | ChatOpenAI(api_key="sk-"),
    get_session_history,  # 세션 기록을 가져오는 함수입니다.
    input_messages_key="input_messages",  # 입력 메시지의 키를 지정합니다.
)

In [24]:
with_message_history.invoke(
    {"input_messages": "코사인의 의미는 무엇인가요?"},
    # 설정 옵션을 딕셔너리 형태로 전달합니다.
    config={"configurable": {"session_id": "xyz123"}},
)

AIMessage(content='코사인은 삼각함수 중 하나로, 직각삼각형에서의 각도에 대해 대변과 빗변의 비율을 나타내는 값입니다. 코사인은 주어진 각도의 삼각비를 나타내며, 삼각형의 변 길이를 통해 삼각형의 각도를 계산하거나, 각도를 통해 삼각형의 변 길이를 계산하는 데 사용됩니다.', response_metadata={'token_usage': {'completion_tokens': 146, 'prompt_tokens': 24, 'total_tokens': 170}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-9f610bcd-f92d-4bf0-add2-e40b580232d4-0', usage_metadata={'input_tokens': 24, 'output_tokens': 146, 'total_tokens': 170})

# 영구 저장소(Persistent storage)

- 영구 저장소(Persistent storage)
    - 프로그램이 종료되거나 시스템이 재부팅되더라도 데이터를 유지하는 저장 메커니즘
    - 데이터베이스, 파일 시스템, 또는 기타 비휘발성 저장 장치를 통해 구현될 수 있음

- 영구 저장소는 애플리케이션의 상태를 저장하고, 사용자 설정을 유지하며, 장기간 데이터를 보존하는 데 필수적.
</br>이를 통해 프로그램은 이전 실행에서 중단된 지점부터 다시 시작할 수 있으며, 사용자는 데이터 손실 없이 작업을 계속 할 수 있음.

- RunnableWithMessageHistory는 get_session_history 호출 가능 객체가 채팅 메시지 기록을 어떻게 검색하는지에 대해 독립적
- 로컬 파일 시스템을 사용하는 예제: https://github.com/langchainai/langserve/blob/main/examples/chat_with_persistence_and_user/server.py 
- 다른 제공자를 사용하여 채팅 메시지 기록을 구현하는 방법은 memory integrations 페이지 확인: https://python.langchain.com/v0.2/docs/integrations/platforms/ 

<Redis를 사용하는 방법>

## Redis 설치

In [26]:
%pip install -qU redis

Note: you may need to restart the kernel to use updated packages.


## Redis 서버 구동

- 기존에 연결할 Redis 배포가 없는 경우, 로컬 Redis Stack 서버 시작
- Docker 로 Redis 서버를 구동하는 명령어
    - docker run -d -p 6379:6379 -p 8001:8001 redis/redis-stack:latest
- REDIS_URL 변수에 Redis 데이터베이스의 연결 URL 할당
    - URL은 "redis://localhost:6379/0"로 설정

In [36]:
#주피터노트북에서 실행
!docker run -d -p 6379:6379 -p 8001:8001 redis/redis-stack:latest

'docker'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.


**-> 도커 설치필요..ㅎ**

In [28]:
# Redis 서버의 URL을 지정합니다.
REDIS_URL = "redis://localhost:6379/0"

**LangSmith 추적 설정**

추적을 위한 LangSmith 설정 </br>
LangSmith는 필수적인 것은 아니지만, 도움이 될 수 있음.

In [31]:
from dotenv import load_dotenv
import os

load_dotenv()

# LANGCHAIN_TRACING_V2 환경 변수를 "true"로 설정합니다.
os.environ["LANGCHAIN_TRACING_V2"] = "true"
# LANGCHAIN_PROJECT 설정
os.environ["LANGCHAIN_PROJECT"] = "RunnableWithMessageHistory"

메시지 기록 구현을 업데이트하려면 새로운 호출 가능한 객체를 정의하고, 이번에는 RedisChatMessageHistory의 인스턴스를 반환하면 됨.

In [33]:
from langchain_community.chat_message_histories import RedisChatMessageHistory


def get_message_history(session_id: str) -> RedisChatMessageHistory:
    # 세션 ID를 기반으로 RedisChatMessageHistory 객체를 반환합니다.
    return RedisChatMessageHistory(session_id, url=REDIS_URL)


with_message_history = RunnableWithMessageHistory(
    runnable,  # 실행 가능한 객체
    get_message_history,  # 메시지 기록을 가져오는 함수
    input_messages_key="input",  # 입력 메시지의 키
    history_messages_key="history",  # 기록 메시지의 키
)

이전과 동일한 방식으로 호출 가능

In [ ]:
with_message_history.invoke(
    # 수학 관련 질문 "코사인의 의미는 무엇인가요?"를 입력으로 전달합니다.
    {"ability": "math", "input": "What does cosine mean?"},
    # 설정 옵션으로 세션 ID를 "redis123" 로 지정합니다.
    config={"configurable": {"session_id": "redis123"}},
)